In [2]:
import pandas as pd

In [3]:
fname='https://www.dropbox.com/scl/fi/bfcdwzqmrvmxzi72qqp5y/diabetes_dataset.csv?rlkey=u729vuphkaxphmuvldsof3x72&st=mte8gpgo&dl=1'
diabetes_raw_df=pd.read_csv(fname)

In [6]:
diabetes_raw_df


,Diabetes_binary,HighBP,BMI,PhysActivity,Sex,Age,Education
0,0,1,26,1,1,4,6
1,0,1,26,0,1,12,6
2,0,0,26,1,1,13,6
3,0,1,28,1,1,11,6
4,0,0,29,1,0,8,5
...,...,...,...,...,...,...,...
70687,1,0,37,0,0,6,4
70688,1,0,29,0,1,10,3
70689,1,1,25,0,0,13,6
70690,1,1,18,0,0,11,2


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Prepare the data
X = diabetes_raw_df.drop('Diabetes_binary', axis=1).values
y = diabetes_raw_df['Diabetes_binary'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Define the neural network model
class DiabetesClassifier(nn.Module):
    def __init__(self):
        super(DiabetesClassifier, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Instantiate the model, define the loss function and the optimizer
model = DiabetesClassifier()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    predictions = predictions.round()
    accuracy = (predictions.eq(y_test).sum() / float(y_test.shape[0])).item()
    print(f'Accuracy: {accuracy:.4f}')


Epoch [10/100], Loss: 0.6963
Epoch [20/100], Loss: 0.6913
Epoch [30/100], Loss: 0.6865
Epoch [40/100], Loss: 0.6809
Epoch [50/100], Loss: 0.6739
Epoch [60/100], Loss: 0.6649
Epoch [70/100], Loss: 0.6534
Epoch [80/100], Loss: 0.6398
Epoch [90/100], Loss: 0.6249
Epoch [100/100], Loss: 0.6097
Accuracy: 0.7073
